<a href="https://www.kaggle.com/code/aayushpurswani/malaria?scriptVersionId=127142480" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#All import Statements 
import numpy as np
np.random.seed(100)

import pandas as pd 
import cv2 
import os 
from PIL import Image 
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [ ]:
os.environ['KERAS_BACKEND'] = 'tensorflow'

In [ ]:
size = 64
dataset=[]
labels=[]

parasitized_images = os.listdir('/kaggle/input/cell-images-for-detecting-malaria/cell_images/Parasitized/')
for i, image_name in enumerate(parasitized_images):
    if(image_name.split('.')[1] == 'png'):
        image = cv2.imread('/kaggle/input/cell-images-for-detecting-malaria/cell_images/Parasitized/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((size,size))
        dataset.append(np.array(image))
        labels.append(0)

uninfected_images = os.listdir('/kaggle/input/cell-images-for-detecting-malaria/cell_images/Uninfected/')
for i, image_name in enumerate(uninfected_image):
    if(image_name.split('.')[1] == 'png'):
        image = cv2.imread('/kaggle/input/cell-images-for-detecting-malaria/cell_images/Uninfected/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((size,size))
        dataset.append(np.array(image))
        labels.append(1)

In [ ]:
input_shape = (size, size, 3)   #change to (SIZE, SIZE, 3)
inp = keras.layers.Input(shape=input_shape)

conv1 = keras.layers.Conv2D(32, kernel_size=(3, 3), 
                               activation='relu', padding='same')(inp)
pool1 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
norm1 = keras.layers.BatchNormalization(axis = -1)(pool1)
drop1 = keras.layers.Dropout(rate=0.2)(norm1)
conv2 = keras.layers.Conv2D(32, kernel_size=(3, 3), 
                               activation='relu', padding='same')(drop1)
pool2 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
norm2 = keras.layers.BatchNormalization(axis = -1)(pool2)
drop2 = keras.layers.Dropout(rate=0.2)(norm2)

flat = keras.layers.Flatten()(drop2)  #Flatten the matrix to get it ready for dense.

hidden1 = keras.layers.Dense(512, activation='relu')(flat)
norm3 = keras.layers.BatchNormalization(axis = -1)(hidden1)
drop3 = keras.layers.Dropout(rate=0.2)(norm3)
hidden2 = keras.layers.Dense(256, activation='relu')(drop3)
norm4 = keras.layers.BatchNormalization(axis = -1)(hidden2)
drop4 = keras.layers.Dropout(rate=0.2)(norm4)

out = keras.layers.Dense(2, activation='sigmoid')(drop4)   #units=1 gives error

model = keras.Model(inputs=inp, outputs=out)
model.compile(optimizer='adam',
                loss='binary_crossentropy',   #Check between binary_crossentropy and categorical_crossentropy
                metrics=['binary_accuracy'])
print(model.summary())

In [ ]:
#Model fitting and training 

X_train, X_valid, y_train, y_valid = train_test_split(dataset, to_categorical(np.array(label)), test_size = 0.2, random_state = 0)
history = model.fit (np.array(X_train), y_train, batch_size = 64 ,epochs = 1,verbose = 1, validation_data = (np.array(X_valid), y_valid))
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot();

model.save('malaria_cnn.h5')